In [62]:
import pandas as pd

La funzione detect_anomalies Restituisce le entry del dataframe che hanno almeno <seconds_threshold> secondi di valori uguali. 

In [63]:
def detect_anomalies(accelerometer_df: pd.DataFrame, hand_type: str, seconds_threshold: int = 5) -> pd.DataFrame:
    """
    Restituisce le entry del dataframe che hanno almeno <seconds_threshold> secondi di valori uguali
    hand_type: D -> Mano dominante, ND -> Mano non dominante
    """
    if hand_type != "D" and hand_type != "ND":
        raise ValueError("hand_type deve essere 'D' o 'ND'")
    
    campionamenti_per_secondo = 8 # l'accelerometro esegue 8 campionamenti al secondo
    seconds_threshold = seconds_threshold*campionamenti_per_secondo

    df_diff = accelerometer_df[[f'x_{hand_type}', f'y_{hand_type}', f'z_{hand_type}']].diff()
    anomalies = df_diff.rolling(window=seconds_threshold).apply(lambda x: all(num == 0 for num in x)) # Recupera le righe che hanno almeno 5 secondi di valori uguali
    anomaly_rows = accelerometer_df[anomalies.any(axis=1)]
    return anomaly_rows

In [64]:
df = pd.read_csv('../raw_data/1_week_RAW.csv', nrows=10000)
detect_anomalies(df, "D", 3)

,datetime,x_D,y_D,z_D,x_ND,y_ND,z_ND
7186,2017-05-04 00:01:29.825000,-0.949,0.020,0.273,-0.957,-0.004,0.273
7187,2017-05-04 00:01:29.837500,-0.949,0.023,0.273,-0.953,-0.004,0.277
7188,2017-05-04 00:01:29.850000,-0.949,0.020,0.277,-0.953,-0.004,0.277
7189,2017-05-04 00:01:29.862500,-0.949,0.023,0.273,-0.953,-0.004,0.273
7190,2017-05-04 00:01:29.875000,-0.949,0.023,0.273,-0.957,0.000,0.277
